# Exercise about Reinforcement Learning
(v.1.1)
![Frozen lake](https://debuggirl.files.wordpress.com/2013/03/img_43081.jpg "Frozen lake")

## The Frozen Lake game
The Frozen Lake game is about safely navigating a frozen lake whose ice happen have several hidden cracks or holes in it, stepping on any of these will result in you falling into the water. The goal is to reach a part of the lake where a hidden treasure is located. Both the weak spots (holes) and the goal are not known beforehand by the player. Here we will train an *agent* to navigate this lake safely and reach the goal. To do so we will be using the **Q-learning** algorithm. Here we use the **Q-value function** $Q({s, a}) = v$ that takes as input the current state $s$ and possible action $a$, and returns its value $v$. The frozen lake is represented as a grid of cells (gridworld), where each cell is associated with a state that can be start `S`, frozen lake `F`, hole in the ice `H`, or goal `G`, and the actions available at each state is to navigate *left*, *down*, *right*, or *up*.<br>
<br>
When playing this game the agent will be iteratively doing the following:
1. Choose an action given the current state of the agent (calculate where to move).
2. Perform the chosen action (move to the chosen cell).
3. Upadete the Q-value function (the Q-table) based on the reward from performing this action.
4. Set the new state of the agent.

In [1]:
# Import the libraries we will be using
import gym
import numpy as np
import time
import random
from IPython.display import clear_output

In [2]:
# Initiate the FrozenLake game
env = gym.make('FrozenLake-v1', is_slippery=False, map_name="4x4")

In [3]:
# Next we render the game board
env.render()


SFFF
FHFH
FFFH
HFFG


After running the above, you should should see the gameboard, i.e. the frozen lake.
<br>
SFFF</br>
FHFH<br>
FFFH<br>
HFFG<br>


It consist of the following cells:
 - S - Start.
 - F - Frozen lake, which is safe to walk on.
 - H - Hole in the ice. Reaching this cell ends the round.
 - G - Goal. Reaching this cell gives +1 point and ends the round.

As mentioned, we will now train an agent to navigate this frozen lake and ultimately reach the goal (`G` cell) with as few steps/turns as possible without falling into the lake (stepping on any of the `H` cells). At start the agent does not know where the goal is or what cells are safe to walk on. However, with repeated trials, combined with the delayed feedback reward gained from stepping on the various cells, the agent should eventually learn to navigate this lake.

In [4]:
state_space_size = env.observation_space.n
action_space_size = env.action_space.n
print('The frozen lake (gameboard) consist of {} states (cells) and {} actions for each of these.'.format(state_space_size, action_space_size))
print('So for each state there are four actions availble:\n0 = ◀️ left\n1 = 🔽 down\n2 = ▶️ right\n3 = 🔼 up')

The frozen lake (gameboard) consist of 16 states (cells) and 4 actions for each of these.
So for each state there are four actions availble:
0 = ◀️ left
1 = 🔽 down
2 = ▶️ right
3 = 🔼 up


### Q-table
We will be using a **Q-table** as the underlying representation of the value function (Q-value function). For this type of game board, the Q-table is a well suited representation to use. This table will store all the knowledge gained by the agent - each *state* will have information about the *value* of taking any of the available *actions* (◀️, 🔽, ▶️, 🔼) when the policy is to find the hidden treasure without falling into the lake. To start with, all the q-values in this table will be zero, but these will be updated as the agent is playing.

In [5]:
q_table = np.zeros((state_space_size, action_space_size))

# Print some info about it
s = 'Q-table:\n'
i = 0
j = 0
for cr in q_table:
    s += '{}\t<-- state [{},{}]\n'.format(cr, i, j)
    j += 1
    if j >= action_space_size:
        j = 0
        i += 1
print(s)

Q-table:
[0. 0. 0. 0.]	<-- state [0,0]
[0. 0. 0. 0.]	<-- state [0,1]
[0. 0. 0. 0.]	<-- state [0,2]
[0. 0. 0. 0.]	<-- state [0,3]
[0. 0. 0. 0.]	<-- state [1,0]
[0. 0. 0. 0.]	<-- state [1,1]
[0. 0. 0. 0.]	<-- state [1,2]
[0. 0. 0. 0.]	<-- state [1,3]
[0. 0. 0. 0.]	<-- state [2,0]
[0. 0. 0. 0.]	<-- state [2,1]
[0. 0. 0. 0.]	<-- state [2,2]
[0. 0. 0. 0.]	<-- state [2,3]
[0. 0. 0. 0.]	<-- state [3,0]
[0. 0. 0. 0.]	<-- state [3,1]
[0. 0. 0. 0.]	<-- state [3,2]
[0. 0. 0. 0.]	<-- state [3,3]



With an empty Q-table the agent do not know anything about what actions are better than others. In other words, the Q-value function will always return zero for every possible action at each state. Thus in the beginning the agent will instead be taking random actions while exploring the board. This is called *exploration*. However, after having gained some knowledge about the board, the agent can start relying on the experience gained and reflected in the Q-value function. This is called *exploitation*. We will be using the *epsilon greedy strategy* to find a balance between *exploration* (random actions) and *exploitation* (optional actions according to the Q-value function).

In [6]:
# Here we introduce some parameters to use in the Q-training algorithm.

# Learning rate (value between 0 and 1)
learning_rate = 0.1

# Reward discount used in the Bellman equation (value between 0 and 1)
discount_factor = 0.99

### The Bellman equation
For updating a state in the Q-table we will here be using the following version of the *Bellman equation*:<br>
$$Q^{updated}(s_{t}, a_{t}) := (1-\alpha)Q(s_{t}, a_{t}) + \alpha (r_{t} + \gamma \underset{a}{\max}Q(s_{t+1},a))$$
$s_{t}$ - current state.<br>
$a_{t}$ - chosen action.<br>
$\alpha$ - learning rate.<br>
$r_{t}$ - reward from taking the action $a_{t}$.<br>
$\gamma$ - discount factor.<br>
$s_{t+1}$ - new state after taking action $a_{t}$.<br>
$\underset{a}{\max}Q(s_{t+1},a)$ - the optional value possible from $s_{t+1}$.<br>

<div class=" alert alert-warning">
    
### Student Task A5.1

<strong><i><u>Your task<i></u></strong> is to implement the Bellman equation as a function that calculates and returns the new value for $Q^{updated}(s_{t}, a_{t})$.<br>It takes as input: Q-table, $s_{t}$, $a_{t}$, $s_{t+1}$, $r_{t}$, $\alpha$, and $\gamma$.
</div>

In [7]:
def q_update(q_table, state, action, new_state, reward, learning_rate, discount_factor):
    """
    Calculate the new value in the q_table using Bellman equation
    """
    ## Implement the Bellman equation as shown above
    # q_new = q_table[state, action] ...
    
    ### BEGIN SOLUTION
    q_new = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_factor * np.max(q_table[new_state, :]))
    ### END SOLUTION
    
    return q_new

In [8]:
# this cell is for tests

### BEGIN HIDDEN TESTS
# Should this test be improved somehow?
assert q_update(q_table=q_table, state=0, action=2, new_state=1, reward=0, learning_rate=learning_rate, discount_factor=discount_factor) == 0.0
assert q_update(q_table=q_table, state=0, action=2, new_state=1, reward=1, learning_rate=learning_rate, discount_factor=discount_factor) == 0.1

test_q_table = np.array([[0.58940116, 0.50458308, 0.50449804, 0.50430707],
                [0.39701686, 0.31072888, 0.32279088, 0.52844835],
                [0.40813145, 0.43028301, 0.41980598, 0.4987701],
                [0.34803986, 0.31594787, 0.31342393, 0.47886813],
                [0.6137157, 0.39051021, 0.33313007, 0.40253577],
                [0.,         0.,         0.,         0.        ],
                [0.45621221, 0.157744,  0.16167418, 0.15956471],
                [0.,         0.,         0.,         0.        ],
                [0.40419588, 0.44695664, 0.49716483, 0.66166421],
                [0.48304007, 0.70040278, 0.43290747, 0.42981004],
                [0.68873522, 0.45787092, 0.36492453, 0.39246048],
                [0.,         0.,         0.,         0.        ],
                [0.,         0.,         0.,         0.        ],
                [0.45736412, 0.66062829, 0.80533251, 0.59763036],
                [0.7685619,  0.9257641,  0.77140422, 0.75617869],
                [0.,         0.,         0.,         0.        ]])
assert q_update(q_table=test_q_table, state=8, action=2, new_state=9, reward=0, learning_rate=learning_rate, discount_factor=discount_factor) == 0.51678822222
### END HIDDEN TESTS

In [9]:
# For the mentioned epsilon greedy strategy we will need a few more parameters.

# Upper bound for the exploration rate (value between 0 and 1)
max_exploration_rate = 1.0

# Lower bound for the exploration rate (value between 0 and 1)
min_exploration_rate = 0.01  # 0.01

# How much the exploration rate decays per episode (down to the min_exploration_rate, value between 0 and 1).
exploration_decay_rate = 0.001

### Epsilon greedy strategy
By using the *epsilon greedy strategy*, we will let the exploration rate start high (1), and then decrease for every episode with exponential decay. The goal here is to find the optimal balance between exploration and exploitation. Each round will consist of 10000 episodes. In the first episodes we want to have a high probility of performing exploration actions to avoid getting stuck in sub-optimal paths (local maximums). However, we want to gradually start leaning more towards exploitation towards the later episodes.<br>
<br>
We will be using the following formula for calculating the exploration rate for a given episode $\epsilon(i)$:<br>
$$\epsilon(i) = \epsilon_{min} + (\epsilon_{max} - \epsilon_{min}) \cdot e^{-\zeta \cdot i}$$
$\epsilon$ - exploration rate.<br>
$i$ - episode number.<br>
$\epsilon_{min}$ - min exploration rate.<br>
$\epsilon_{max}$ - max exploration rate.<br>
$\zeta$ - exploration decay rate.<br>

<div class=" alert alert-warning">
    
### Student Task A5.2

<strong><i><u>Your task<i></u></strong> to implement the `calc_exploration_rate` function which calculates and returns the exploration rate using the above formula.
</div>

In [10]:
# Function that calculate the exploration rate for a given episode
def calc_exploration_rate(episode, min_exploration_rate, max_exploration_rate, exploration_decay_rate):
    ## Calculate the exploration rate
    # exploration_rate = ...
    
    ### BEGIN SOLUTION
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
    ### END SOLUTION
    return exploration_rate

In [11]:
# this cell is for tests

### BEGIN HIDDEN TESTS
assert calc_exploration_rate(700, 0.01, 1, 0.005) == 0.039895409588095315
assert calc_exploration_rate(100, 0.01, 1, 0.005) == 0.6104653531155071
### END HIDDEN TESTS

<div class=" alert alert-warning">
    
### Student Task A5.3

Next we will make the function for choosing an action given a state. This function includes an *exploration rate threshold* which is a randomly drawn number between 0 and 1. For every action, this thershold makes the agent to perform an exploration action with a probability equal to the given *exploration rate*.
<br>
<strong><i><u>Your task<i></u></strong> is to complete the below function `choose_action`. This function takes as input the Q-table, $s_{t}$, and exploration rate.
</div>

In [12]:
def choose_action(q_table, state, exploration_rate):
    ## Pick a random number between 0 and 1
    # exploration_rate_threshold = ...
    
    ### BEGIN SOLUTION 
    exploration_rate_threshold = np.random.uniform(0, 1)
    ### END SOLUTION
    
    
    ## Exploration vs exploitation
    # if exploration_rate > exploration_rate_threshold or sum(q_table[state, :]) == 0:
        ## Exploration, i.e., random action        
        # action = ...
    # else:
        ## Exploitation, i.e., perform the optimal action according to the Q-table
        # action = ...
    
    ### BEGIN SOLUTION
    if exploration_rate > exploration_rate_threshold or sum(q_table[state, :]) == 0:
        ## Exploration, i.e., random action 
        action = random.choice(range(len(['left', 'down', 'right', 'up'])))
        # We could also use: action = env.action_space.sample()
    else:
        ## Exploitation, i.e., perform the optimal action according to the Q-table
        action = np.argmax(q_table[state, :])
    ### END SOLUTION

    return action

In [13]:
# this cell is for tests

### BEGIN HIDDEN TESTS
test_q_table = np.array([[0.58940116, 0.50458308, 0.50449804, 0.50430707],
                [0.39701686, 0.31072888, 0.32279088, 0.52844835],
                [0.40813145, 0.43028301, 0.41980598, 0.4987701],
                [0.34803986, 0.31594787, 0.31342393, 0.47886813],
                [0.6137157, 0.39051021, 0.33313007, 0.40253577],
                [0.,         0.,         0.,         0.        ],
                [0.45621221, 0.157744,  0.16167418, 0.15956471],
                [0.,         0.,         0.,         0.        ],
                [0.40419588, 0.44695664, 0.49716483, 0.66166421],
                [0.48304007, 0.70040278, 0.43290747, 0.42981004],
                [0.68873522, 0.45787092, 0.36492453, 0.39246048],
                [0.,         0.,         0.,         0.        ],
                [0.,         0.,         0.,         0.        ],
                [0.45736412, 0.66062829, 0.80533251, 0.59763036],
                [0.7685619,  0.9257641,  0.77140422, 0.75617869],
                [0.,         0.,         0.,         0.        ]])

action_list = []
for i in range(100):
    i_action = choose_action(test_q_table, 3, 0)
    action_list.append(i_action)
assert action_list[0] == 3 and all(x == action_list[0] for x in action_list)

action_list = []
for i in range(100):
    i_action = choose_action(test_q_table, 3, 1)
    action_list.append(i_action)
assert not all(x == action_list[0] for x in action_list)
### END HIDDEN TESTS

### Q-Learning algorithm
Now it is time to train our agent using the Q-learning algorithm as shown below.
If everything works as intended, the below code should print some statistics from the training.

In [14]:
# Number of episodes to train
num_episodes = 10000

# Max number of steps before ending the round
max_steps_per_episode = 100

In [15]:
# Q-Learning algorithm
rewards_all_episodes = []
steps_all_episodes = []
exploration_rate = max_exploration_rate

# Play the game for n episodes
for episode in range(num_episodes):
    state = env.reset()
    done = False
    rewards_current_episode = 0
    steps_current_episode = 0
    
    for step in range(max_steps_per_episode):
        # Chooce an action. Exploration-exploitation trade-off with the epsilon greedy strategy
        action = choose_action(q_table, state, exploration_rate)
        
        # Perform the chosen action
        new_state, reward, done, info = env.step(action)
        
        # Upadete the Q-table
        q_table[state, action] = q_update(q_table, state, action, new_state, reward, learning_rate, discount_factor)
        
        # Set the new state
        state = new_state
        
        # Collect some statistics
        rewards_current_episode += reward
        steps_current_episode += 1
        
        if done == True:
            break

    # Update the exploration rate
    exploration_rate = calc_exploration_rate(episode, min_exploration_rate, max_exploration_rate, exploration_decay_rate)

    rewards_all_episodes.append(rewards_current_episode)
    steps_all_episodes.append(steps_current_episode)


# Print the average rewards
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
steps_per_thousand_episodes = np.split(np.array(steps_all_episodes), num_episodes/1000)

print('Training results, average reward and steps per thousand episodes:')
count = 1000
for i, _ in enumerate(rewards_per_thousand_episodes):
    r = rewards_per_thousand_episodes[i]
    s = steps_per_thousand_episodes[i]
    print('{}: rewards {:.4}, steps {:.4}'.format(count, sum(r/1000), sum(s/1000)))
    count += 1000

Training results, average reward and steps per thousand episodes:
1000: rewards 0.257, steps 8.017
2000: rewards 0.734, steps 6.831
3000: rewards 0.905, steps 6.326
4000: rewards 0.945, steps 6.091
5000: rewards 0.975, steps 6.072
6000: rewards 0.983, steps 6.032
7000: rewards 0.987, steps 6.029
8000: rewards 0.989, steps 6.027
9000: rewards 0.992, steps 6.044
10000: rewards 0.99, steps 6.044


In [16]:
print('Trained Q-table:')
print(q_table)

Trained Q-table:
[[0.94148015 0.93206535 0.95099005 0.94148015]
 [0.94148015 0.         0.96059601 0.95099005]
 [0.95099005 0.970299   0.95099005 0.96059601]
 [0.96059601 0.         0.86161114 0.9060303 ]
 [0.88425553 0.75878929 0.         0.94148015]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.         0.96059597]
 [0.         0.         0.         0.        ]
 [0.3574754  0.         0.93396316 0.48765719]
 [0.58939914 0.89163075 0.98009975 0.        ]
 [0.97029515 0.99       0.         0.97029897]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.75452119 0.98999972 0.72104627]
 [0.98009562 0.98999955 1.         0.98009987]
 [0.         0.         0.         0.        ]]


### Playing Frozen Lake
Finally we will let the agent play Frozen Lake using the Q-table that we have learned above.

<div class=" alert alert-warning">
    
### Student Task A5.4

Now we are ready to put the agent to navigate the frozen lake based on what it has learned. Maybe you can, by looking at the Q-table already see what route the agent will be taking?<br><br>
<strong><i><u>Your task<i></u></strong> is to run the below code and observe how the agent is now navigating the frozen lake. Observe the learned Q-table to understand why the agent is taking this specific path.
</div>

In [18]:
# Watch the agent play for a few rounds using the learned Q-table
n_rounds = 3
for episode in range(n_rounds):
    state = env.reset()
    done = False

    print('༺❁ EPISODE {} ❁༻'.format(episode+1))
    time.sleep(1)

    for step in range(max_steps_per_episode + 1):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)

        action = np.argmax(q_table[state, :])

        new_state, reward, done, info = env.step(action)

        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print('༺❁ You reached the goal after {} steps! ❁༻'.format(step + 1))
                time.sleep(3)
            elif step == max_steps_per_episode - 1:
                print('༺❁ No steps left ({} steps max)! ❁༻'.format(step + 1))
                time.sleep(3)
            else:
                print('༺❁ You fell through a hole after {} steps! ❁༻'.format(step + 1))
                time.sleep(3)
            clear_output(wait=True)
            break
        elif step == max_steps_per_episode - 1:
            print('༺❁ No steps left ({} steps max)! ❁༻'.format(step + 1))
            time.sleep(3)
            clear_output(wait=True)
            break
        state = new_state
time.sleep(3)
print('༺❁ Done ❁༻')
env.close()

༺❁ Done ❁༻


### The ice can get slippery!
![Slippery ice](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS7iaMa5OcNHtJFKB1fZo_EvG-nQyPPTm_fsA&usqp=CAU "Slippery ice")

So far the ice has been easy to walk on, and we have seen that the agent has not had any problem in finding an optimal path leading to the goal. Now, however, the weather has changed and the ice has suddenly become very slippery.
In the next part of this exercise we want you to go to the top/beginning of this notebook to the cell where we initate the Frozen Lake environment:<br>
`env = gym.make('FrozenLake-v1', is_slippery=False, map_name="4x4")`<br>
Now change `is_slippery=False` to `is_slippery=True`.<br>
Setting `is_slippery=True` means that there is only a 1/3 chance that the agent will move in the intended 
direction, and 2/3 chance to move in one of the perpendicular directions (1/3 and 1/3). 
E.g., if **action = left**, then:
- P(move left) = 1/3
- P(move up) = 1/3
- P(move down) = 1/3

Next, re-run the whole notebook (▶▶) and observe how the new trained Q-table looks like, how the new statistics has become, and also observe how the behaviour of the agent has changed at run time.

<div class=" alert alert-warning">
    
### Student Task A5.5

First make sure to set `is_slippery=True` at the top/beginning of this notebook.<br>
<strong><i><u>Your task<i></u></strong> is to briefly explore the impact of different values assigned to the different parameters, and try to find the most promising values that optimize the reward gained by the agent (look at the scores from the last 1000 episodes).
The parameters in question are:
- learning_rate ($\alpha$)
- discount_factor ($\gamma$)
- max_exploration_rate ($\epsilon_{max}$)
- min_exploration_rate ($\epsilon_{min}$)
- exploration_decay_rate ($\zeta$)

<u>Be prepared to discuss your findings in the exercise lecture.</u><br>
Some additional things to think about: Why is the preferred first move of the agent to move left? Can you think of some possible approaches and strategies that could improve the performance of the agent further? Can you think of some completely different ways for how to solve this game/problem?
</div>